## 🔹 Concepto
Una **Convolutional Neural Network (CNN)** es una red neuronal especializada en datos con estructura espacial, como imágenes.  
Las CNN aprenden automáticamente **características jerárquicas** mediante:
- **Convoluciones:** filtros que extraen patrones locales (bordes, texturas).  
- **Pooling:** reducción de dimensionalidad y estabilidad frente a traslaciones.  
- **Capas fully connected:** combinan las características para producir la salida final.

En esta sección entrenaremos una **CNN desde cero** sobre el dataset **CIFAR-10** (imágenes 32x32, 10 clases).

In [4]:
# 🔹 Importar librerías y preparar dataset CIFAR-10
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device('cpu')

# Transformaciones: normalización + aumento simple
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


## 🔹 Definición de la CNN
La arquitectura básica incluye:
- 2 capas convolucionales con ReLU + MaxPooling
- Capa fully connected con Dropout para prevenir overfitting
- Softmax implícito en CrossEntropyLoss

# 🔹 Convolutional Neural Networks (CNNs) – Explicación completa

Una **CNN (Convolutional Neural Network)** es un tipo de red neuronal especializada en datos con estructura de grilla, como imágenes, y está diseñada para aprender **representaciones jerárquicas** mediante una combinación de capas convolucionales, funciones de activación, pooling, fully connected y técnicas de regularización como Dropout y Batch Normalization. La idea principal es extraer automáticamente características relevantes de las imágenes sin necesidad de ingeniería manual de features.

Las **capas convolucionales** (`nn.Conv2d`) aplican filtros (kernels) sobre la imagen de entrada para capturar patrones locales como bordes, texturas o formas. Cada filtro genera un **mapa de activación**, y la cantidad de filtros se define por el parámetro `out_channels`. Los parámetros principales de una convolución son `in_channels` (canales de entrada, por ejemplo 3 para RGB), `out_channels` (número de filtros), `kernel_size` (tamaño del filtro, por ejemplo 3x3), `stride` (paso de desplazamiento del filtro) y `padding` (relleno de ceros alrededor de la imagen para mantener dimensiones). La salida de una convolución es un tensor de forma `[batch_size, out_channels, H_out, W_out]`.

Después de la convolución se aplica una **función de activación** para introducir no linealidad en el modelo. Las más comunes son: **ReLU**, que toma el máximo entre cero y la entrada, evitando gradientes desaparecientes; **Sigmoid**, útil para probabilidades, pero que puede saturar en valores extremos; y **Tanh**, que centra la salida entre -1 y 1, aunque también puede saturar los gradientes.

Luego se suelen aplicar capas de **pooling**, generalmente `MaxPool2d`, que reducen las dimensiones espaciales del mapa de activaciones conservando la información más relevante. Esto disminuye la complejidad del modelo, hace que la red sea más resistente a traslaciones y reduce la cantidad de parámetros en las capas posteriores. Por ejemplo, un `MaxPool2d(2)` reduce la altura y anchura a la mitad tomando el valor máximo de cada bloque 2x2.

Después de varias capas de convolución + activación + pooling, los mapas de activación se **aplanan** (`x.view(x.size(0), -1)`) para ser procesados por capas **fully connected** (`nn.Linear`). Estas capas conectan todas las neuronas de la entrada con todas las neuronas de la salida, combinando las características extraídas por las convoluciones para generar la predicción final. Los parámetros de `nn.Linear` son `in_features` (número de entradas, normalmente la salida aplanada) y `out_features` (número de salidas, por ejemplo el número de clases).

Para mejorar la **generalización y evitar overfitting**, se suele usar **Dropout** (`nn.Dropout(p)`), que apaga aleatoriamente un porcentaje `p` de neuronas durante el entrenamiento. Esto fuerza a la red a no depender excesivamente de ciertas neuronas y distribuye la información entre todas.


# 🔹 Explicación detallada de cada capa

### 1️⃣ nn.Conv2d(3, 32, kernel_size=3, padding=1)
- **Propósito:** capa convolucional que extrae características locales de la imagen.
- **Parámetros:**
  - `3`: número de canales de entrada. Ejemplo: 3 para imágenes RGB.
  - `32`: número de filtros (o mapas de activación) que se aplicarán, es decir, cuántas características distintas extraerá esta capa.
  - `kernel_size=3`: tamaño del filtro (3x3). Este filtro se mueve sobre la imagen aplicando un producto punto local.
  - `padding=1`: agrega un borde de ceros alrededor de la imagen para mantener las dimensiones (H y W) iguales después de la convolución.

- **Salida:** si la imagen de entrada era 32x32x3, la salida será 32x32x32 (altura x anchura x filtros).

---

### 2️⃣ nn.ReLU()
- **Propósito:** función de activación no lineal.
- **Qué hace:** reemplaza todos los valores negativos por 0.  
- **Por qué se usa:** introduce no linealidad y evita el problema de gradientes desaparecidos en redes profundas.

---

### 3️⃣ nn.MaxPool2d(2)
- **Propósito:** capa de *pooling* para reducir las dimensiones espaciales.
- **Parámetros:**
  - `2`: tamaño de la ventana de pooling (2x2).  
- **Qué hace:** divide la entrada en bloques 2x2 y toma el valor máximo de cada bloque.
- **Efecto:** reduce la altura y anchura a la mitad (por ejemplo, de 32x32 a 16x16), manteniendo la profundidad (número de filtros).

---

### 4️⃣ nn.Conv2d(32, 64, kernel_size=3, padding=1)
- **Propósito:** segunda capa convolucional que extrae características más complejas.
- **Parámetros:**
  - `32`: canales de entrada (provenientes de la salida de la capa anterior).
  - `64`: número de filtros, aumentando la capacidad de aprender características más abstractas.
  - `kernel_size=3`: tamaño del filtro (3x3).
  - `padding=1`: mantiene la dimensión espacial igual.

- **Salida:** si la entrada era 16x16x32, la salida será 16x16x64.

---

### 5️⃣ nn.ReLU()
- Igual que la primera ReLU, aplica no linealidad.

---

### 6️⃣ nn.MaxPool2d(2)
- Igual que la primera capa de pooling.
- Reduce las dimensiones de 16x16 a 8x8, manteniendo los 64 mapas de activación.


El **flujo completo** de una CNN típica para imágenes 32x32x3 (como CIFAR-10) sería:

1. Entrada: Imagen 32x32x3.
2. Conv2d(3→32, 3x3) + ReLU → extracción de características locales.
3. MaxPool2d(2) → reducción de dimensión espacial a 16x16x32.
4. Conv2d(32→64, 3x3) + ReLU → extracción de características más complejas.
5. MaxPool2d(2) → reducción a 8x8x64.
6. Conv2d(64→128, 3x3) + ReLU → características de alto nivel.
7. MaxPool2d(2) → reducción a 4x4x128.
8. Flatten → tensor de 2048 neuronas (4*4*128).
9. Linear(2048→256) + ReLU → combinación de características.
10. Dropout(p=0.5) → regularización.
11. Linear(256→10) → salida final de clases.

Durante el **forward pass**, la imagen pasa por todas estas capas generando activaciones que representan cada vez niveles más abstractos de la información. Luego se calcula la **pérdida** comparando la predicción con la verdad (por ejemplo, `CrossEntropyLoss`). En el **backward pass**, se calculan los gradientes mediante **backpropagation**, y un **optimizador** (Adam, SGD, RMSProp) actualiza los pesos. El entrenamiento se realiza por **épocas**, mejorando iterativamente la capacidad de la red para generalizar.

Opcionalmente, se puede usar **Batch Normalization** para normalizar las activaciones dentro de cada mini-batch, estabilizando y acelerando el entrenamiento. También se puede aplicar **gradient clipping** para limitar el tamaño de los gradientes y evitar explosiones.

Las CNNs son muy efectivas en **clasificación, detección de objetos, segmentación de imágenes y reconocimiento de patrones**. Además, se pueden usar en **Transfer Learning**, aprovechando modelos preentrenados como ResNet o VGG y reemplazando solo la última capa para adaptarlos a un nuevo dataset, reduciendo tiempos de entrenamiento y mejorando performance.

En resumen, una CNN combina convoluciones, activaciones, pooling, fully connected y regularización para aprender representaciones jerárquicas de imágenes de manera eficiente, y cada componente influye en la capacidad de aprendizaje, velocidad de entrenamiento y generalización del modelo.


In [7]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(64*8*8, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

cnn_model = SimpleCNN().to(device)


## 🔹 Entrenamiento de la CNN
Usamos **Adam** como optimizador y **CrossEntropyLoss** para clasificación múltiple.  
Entrenaremos por unas pocas épocas para demostración.

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

def train(model, loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            _, predicted = y_pred.max(1)
            correct += predicted.eq(y).sum().item()
            total += y.size(0)
        acc = correct / total
        print(f"Epoch {epoch+1} | Loss: {total_loss/len(loader):.4f} | Accuracy: {acc:.4f}")

train(cnn_model, train_loader, epochs=10)


Epoch 1 | Loss: 1.1488 | Accuracy: 0.5928
Epoch 2 | Loss: 1.1182 | Accuracy: 0.6062
Epoch 3 | Loss: 1.0852 | Accuracy: 0.6189
Epoch 4 | Loss: 1.0630 | Accuracy: 0.6255
Epoch 5 | Loss: 1.0428 | Accuracy: 0.6320
Epoch 6 | Loss: 1.0285 | Accuracy: 0.6372
Epoch 7 | Loss: 1.0089 | Accuracy: 0.6456
Epoch 8 | Loss: 1.0002 | Accuracy: 0.6508
Epoch 9 | Loss: 0.9865 | Accuracy: 0.6576
Epoch 10 | Loss: 0.9761 | Accuracy: 0.6585
